In [1]:
from Sim_all_item import *


import collections

import pdb
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import time
from tqdm import tqdm

reward: 0 -1 1 10


/home/zlxlekta924/anaconda3/envs/test/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



# DQN Agent

In [2]:
'''
Architecture of Layers

Dqn 에서의 구조
84x84x4 사이즈
순서대로 input, channel, kernel, strdides
Conv2d(x, 32, 8, 4)
conv2d(x, 64, 4, 2)
conv2d(x, 64, 3, 1)
flatten()
dense(x, 512)
dense(x, 2)

이 때 우리는
9x10x1 이고 약 300배 차이가 나므로

Conv2d(x, 16, 4, 2)
conv2d(x, 32, 3, 1)
flatten()
dense(x, 128)
dense(x, 4)

로 변경하면 괜찮을 듯
'''

#뱀게임 참고


class QnetCNN(nn.Module):
    def __init__(self):
        super(QnetCNN, self).__init__()
        

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels= 32, kernel_size=3, stride=1),
            nn.ReLU())

        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, stride=1),
            nn.ReLU())
    
        # Flatten
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(960 , 256)
        self.fc2 = nn.Linear(256, 4)
        self.dropout = nn.Dropout(0.1)

        
    def forward(self,x):
        
        x = torch.reshape(x, (-1, 4, 10, 9)).to(device)

        #pdb.set_trace()
        x = self.layer1(x)
        x = self.dropout(x)
        x = self.layer2(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)# 4개의 값을 반환
        return x

    def sample_action(self, obs, epsilon):
        out = self.forward(obs) 
        coin = random.random() 
        

        if coin < epsilon:  # 동전던지기, eps 확률로 랜덤하게 행동
            return random.randint(0,3)
        else:
            return out.argmax().item() #Action state 값이 더 높은 index 값 호출


def train(q, q_target, memory, optimizer): 
    for i in range(10):
        s, a, r, s_prime, done_mask = memory.sample(batch_size) #32개를 버퍼에서 뽑아 모아 놓은 s,a,r,s_prime,done_mask

        q_out = q(s)                                   # s 값으로 다음 각 action 값들의 value 값 반환
        q_a = q_out.gather(1,a.to(device))         #선택한 액션값들의 q(s,a) 반환
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1) # 다음 state의 각 q(s,a) 값 반환
        target = torch.tensor(r, device=device) + torch.tensor(gamma, device=device) * max_q_prime * torch.tensor(done_mask, device=device) # 배열 맞춰주기, 쓰러진 경우는 제거
        loss = nn.MSELoss()(q_a, target) # DQN 의 손실함수 계산 L1 유클리드

        optimizer.zero_grad() # optimizer 의 모든 parameter 를 0으로 변환
        loss.backward() # loss 에 대한 gradient 계산
        optimizer.step() # 손실값을 바탕으로 Qnet 의 파라미터 업데이트

def model_save(model_dict, opt_dict, epi):
    PATH = './weights/'
    torch.save({
            'model': model_dict,
            'optimizer': opt_dict,
            'epi': epi
            }, PATH + 'all.tar')

## Replay Buffer

In [3]:
class ReplayBuffer():
    
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)

    def put(self, transition):
        self.buffer.append(transition)

    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []

        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch. float), torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
        
    def size(self):
        return len(self.buffer)


## Step

In [6]:
import collections
from Sim import *


learning_rate = 0.0001
gamma = 0.99
buffer_limit  = 100000
batch_size = 128

USE_CUDA = torch.cuda.is_available()
device = torch.device('cuda:0' if USE_CUDA else 'cpu')

env = Simulator()

memory = ReplayBuffer()
q = QnetCNN().to(device)
q_target = QnetCNN().to(device)
q_target.load_state_dict(q.state_dict()) # 현재 Qnet 의 파라미터를 q_target 에 load

optimizer = optim.RMSprop(q.parameters(), lr = learning_rate) # loss 값을 바탕으로 업데이트할 비율 (q_target 말고 q 만 업데이트)

print_interval = 100
score = 0.0


files = pd.read_csv("./data/factory_order_train.csv")
    
    
for epi in tqdm(range(120000)): 

    if epi%100 == 0:
        model_save(q.state_dict(), optimizer.state_dict(), epi)
        
    items = list(files.iloc[0])[0]             #     [epi%39990])[0]
    
    epsilon = max(0.1, 0.50 - 0.02 * (epi//200))

    s = env.reset(0)
    
    done = False
    first = True
    a_step = 0
    
    while not done:
        
        # Action choose
        if first:
            a = 0
            first = False
        
        else:
            a = q.sample_action(torch.from_numpy(s).float(), epsilon) 
            
        grid, s_prime, r, cumul, done, goal_reward = env.step(a)

        s = s_prime
        
        # Calculate
        done_mask = 0.0 if done else 1.0
        memory.put((s,a,r, s_prime, done_mask))
        
        score += r
        a_step +=1
        
        
        if a_step > 500:
            break
  
        
        if done :
            break
            
    if memory.size() > 4000:
        train(q, q_target, memory, optimizer)
        
        
    if epi%print_interval==0 and epi != 0: 
        action = ['↑', '↓', '←', '→'][a]
        print(f'{epi} 에서 챙기는 아이템 : {items}\n마지막 행동 : {action}\n 그 때의 마지막 상태 :\n{grid}\n마지막 보상 : {r}\n 총 받은 보상 :{cumul}')
        q_target.load_state_dict(q.state_dict()) #q_target 지금걸로 업데이트
        print(f"score = {int(score/print_interval)}, n_buffer : {memory.size()}, eps : {epsilon*100}")
        score = 0.0
        
        
    

  0%|                                                                   | 29/120000 [00:23<33:15:22,  1.00it/s]

363번 시도 끝에 성공,,, 고생했다..


  0%|                                                                   | 48/120000 [00:41<33:07:56,  1.01it/s]

365번 시도 끝에 성공,,, 고생했다..


  0%|                                                                   | 52/120000 [00:45<32:36:08,  1.02it/s]

...

445번 시도 끝에 성공,,, 고생했다..


  3%|██                                                             | 4001/120000 [1:38:25<29:35:12,  1.09it/s]

4000 에서 챙기는 아이템 : ['H', 'L', 'M']
마지막 행동 : ←
 그 때의 마지막 상태 :
[[0 0 0 0 0 0 0 3 0]
 [1 1 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1 0]
 [0 1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1 0]
 [0 1 0 1 0 1 0 1 0]
 [1 1 0 1 0 1 0 1 1]
 [1 1 1 1 1 1 1 1 1]
 [1 1 2 1 1 1 1 1 1]
 [0 0 0 0 1 0 0 0 0]]
마지막 보상 : 0
 총 받은 보상 :-13
score = -18, n_buffer : 100000, eps : 10.0


  3%|██                                                             | 4033/120000 [1:38:55<47:24:43,  1.47s/it]


KeyboardInterrupt: 